In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fiona
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import numpy as np

In [2]:
# data import
dataDir = r'C:\Users\DUANYUEYUN\Documents\GRID3\Health facilities\Data\SLE'
df = gpd.read_file(dataDir + '\\Original Data\\' + 'SLE_Health_facilities.gdb', 
                     driver='FileGDB', layer= 'SLE_ISS_Health_Facilities')

In [3]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None 

In [4]:
# Define some variables
country_col = 'countries'
name_col = 'name_of_facility_visited'
type_col = 'type_of_facility_visited'

# 1. What is the list of fields?

In [5]:
print("The fields are", df.columns)

The fields are Index(['starttime', 'endtime', 'today', 'deviceid', 'date_of_visit', 'ist',
       'countries', 'states', 'districts', 'wards', 'designation',
       'name_of_facility_visited', 'name_of_facility_visited_other',
       'type_of_facility_visited', 'priority_level', 'facility_fp_acs',
       'joint_supervision', 'supervised_by_dsno', 'supervised_by_who',
       'current_feed_back', 'surveillance_fp_training', 'case_def_copy_copy1',
       'facility_fp_acs_copy', 'physiotherapy_unit', 'long', 'lat', 'F27',
       'F28', 'F29', 'F_index', 'F_parent_table_name', 'F_parent_index',
       'F_tags', 'F_notes', 'F_version', 'F_duration', 'F_submitted_by',
       'F_xform_id', 'longitude', 'latitude', 'Weeknumbers',
       'YearofActiveSurv', 'geometry'],
      dtype='object')


# 2. What does each describe?

- `starttime`: start time.
- `endtime`: end time.
- `today`: today's date and day of week
- `deviceid`
- `date_of_visit`
- `ist`: region with respect to the continent, unique values include West, Central and East South.
- `countries`
- `states` 
- `districts`
- `wards`: administrative division.
- `designation`: title.

- `name_of_facility_visited`
- `name_of_facility_visited_other`: another name the facility might have.
- `type_of_facility_visited`
- `priority_level` 
- `facility_fp_acs`: focal person access (inferred), values could be 'Y', 'NA', 'N' or missing.
- `joint_supervision`: whether the facility is jointly supervised. 'Y' or 'N'.
- `supervised_by_dsno`: last time the facility was supervised by DSNO. Values could include 'Not in the past 1 year', 'more than 6 months', 'more than 3 months' and so on.
- `supervised_by_who`: last time the facility was supervised by DSNO. Values could include 'Not in the past 1 year', 'more than 6 months', 'more than 3 months' and so on.

- `current_feed_back`: 'Y' or 'N'
- `surveillance_fp_training`: when training is given to focal person or there is no training or focal person is not assigned.
- `case_def_copy_copy1`: ?
- `facility_fp_acs_copy`: ?
- `physiotherapy_unit`: whether there is a physiotherapy unit.
- `long`: longitude
- `lat`: latitude.
- `F27`: ?
- `F28`: ?
- `F29`: almost 100% missing 
- `F_index` 
- `F_parent_table_name`: almost 100% missing 
- `F_parent_index`
- `F_tags`: almost 100% missing 
- `F_notes`: almost 100% missing 
- `F_version`
- `F_duration`
- `F_submitted_by`: almost 100% missing 
- `F_xform_id`
- `longitude` 
- `latitude` 
- `Weeknumbers`: week number in the year
- `YearofActiveSurv`: year
- `geometry`: contains the geometric object

In [6]:
# Examine unique values of certain columns
cols = ['ist', 'facility_fp_acs', 'joint_supervision', 'supervised_by_dsno',
       'current_feed_back', 'surveillance_fp_training', 'case_def_copy_copy1',
       'facility_fp_acs_copy', 'physiotherapy_unit']
for col in cols:
    n_unique = df[col].nunique()
    
    if n_unique < 20:
        print("Unique values of column", col + ':', df[col].unique())
        print('\n')
    
    else:
        print("Column", col, "has too many unique values.")
        print("Example values:", df[col].unique()[:5])
        print('\n')

Unique values of column ist: ['IST_WEST']


Unique values of column facility_fp_acs: ['N' 'Y' 'NA']


Unique values of column joint_supervision: ['Y' 'N']


Unique values of column supervised_by_dsno: ['1_2_WEEKS' 'MORE_THAN_6_MONTHS' 'NOT_IN_THE_PAST_1_YEAR'
 'MORE_THAN_3_MONTHS' 'LESS_THAN_1_WEEK' '2_3_MONTHS' '1_MONTH']


Unique values of column current_feed_back: ['Y' 'N']


Unique values of column surveillance_fp_training: ['1_2_YRS' 'NOT_TRAINED' 'NO_FOCAL_PERSON_ASSIGNED' '6_12_MONTHS'
 'MORE_THAN_2_YRS' '6_MONTHS']


Column case_def_copy_copy1 has too many unique values.
Example values: ['AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGITIS NNT' 'NONE'
 'AFP MEASLES YELLOW_FEVER' 'MEASLES YELLOW_FEVER NNT' 'NOT_APPLICABLE']


Unique values of column facility_fp_acs_copy: ['N' 'NA' 'Y']


Unique values of column physiotherapy_unit: ['N' 'Y']




# 3. Are they all populated?

Most columns have less than 1% missing values. Some columns have more than 50% missing and a few columns have almost 100% missing values.

In [7]:
print("NA values by column")
pd.DataFrame({'count':df.isna().sum(), 'percentage':round(df.isna().sum() / df.shape[0],3) * 100})

NA values by column


,count,percentage
starttime,0,0.0
endtime,5,0.2
today,0,0.0
deviceid,63,2.7
date_of_visit,0,0.0
ist,0,0.0
countries,0,0.0
states,0,0.0
districts,0,0.0
wards,38,1.7


# 4. Is there information on when the data was collected?  If so, what is the date range?

Time related columns are `starttime`, `endtime`, `today`, `date_of_visit`, `Weeknumbers`, `YearofActiveSurv`.

`starttime`, `endtime`, `today`, `date_of_visit` all have strange values. The start and end dates are listed below. `Weeknumbers`, `YearofActiveSurv` seem to be within a reasonable range. Some data cleaning is done to choose one of `today`, `date_of_visit` as the correct date. If both are incorrect, the date is left as na.

In [8]:
# For selecting the time related columns 
cols = ['starttime','endtime','today_dt', 'date_of_visit_dt','Weeknumbers','YearofActiveSurv', 
        'today_wk_num', 'dov_wk_num']

## `today`

In [9]:
# change time related columns to datetime format
df['today_dt'] = pd.to_datetime(df['today'])
print('Column: today')
print("start:", df['today_dt'].min().date(), 
      "\nend:", df['today_dt'].max().date())

Column: today
start: 1970-01-01 
end: 2020-05-18


## `date_of_visit`

In [10]:
# change time related columns to datetime format
df['date_of_visit_dt'] = pd.to_datetime(df['date_of_visit'])
print('Column: date_of_visit')
print("start:", df['date_of_visit_dt'].min().date(), 
      "\nend:", df['date_of_visit_dt'].max().date())

Column: date_of_visit
start: 1900-02-11 
end: 2021-06-16


## `Weeknumbers`

In [11]:
df['Weeknumbers_new'] = df['Weeknumbers'].astype(int)
print('Column: Weeknumbers')
print('min value:', min(df['Weeknumbers_new']))
print('max value:', max(df['Weeknumbers_new']))

Column: Weeknumbers
min value: 3
max value: 52


## `YearofActiveSurv`

In [12]:
df['YearofActiveSurv_new'] = df['YearofActiveSurv'].astype(int)
print('Column: YearofActiveSurv')
print("start:", df['YearofActiveSurv_new'].min(), 
      "\nend:", df['YearofActiveSurv_new'].max())

Column: YearofActiveSurv
start: 2017 
end: 2020


## date cleaning

In [13]:
# create a new 'date' column
dates = []
for idx, row in df.iterrows():
    dov = row['date_of_visit_dt']
    today = row['today_dt']
    
    # if 'date_of_visit' is within the time frame, take 'date_of_visit' as date
    if dov >= pd.to_datetime('2017-01-01') and dov < pd.to_datetime('2020-07-01'):
        dates.append(dov)
    
    # if 'today' is within the time frame, take 'today' as date
    elif today >= pd.to_datetime('2017-01-01') and today < pd.to_datetime('2020-07-01'):
        dates.append(today)
    
    # otherwise, date is na
    else:
        dates.append(np.nan)
        
df['date'] = dates

In [14]:
print("Number of NAs as a result:", df['date'].isna().sum())

Number of NAs as a result: 1


In [15]:
print("Date range after cleaning:")
print("start date:", df['date'].min().date(), 
      "\nend date:", df['date'].max().date())

Date range after cleaning:
start date: 2017-01-20 
end date: 2020-05-18


# 5. How many data points have been collected overall?

In [16]:
print("Number of data points collected:", df.shape[0])

Number of data points collected: 2300


# 6. How many of the facility names are empty, both null or no text?

In [17]:
print("Number of null values:", df[name_col].isna().sum())

Number of null values: 78


In [18]:
print("Number of no text values:", sum(df[name_col] == ""))

Number of no text values: 0


In [19]:
print("Number of data points:", df.shape[0])
print("Number of unique facility names:", df[name_col].nunique())

Number of data points: 2300
Number of unique facility names: 1434


# 7. What are the “types” used? 
Each health Facility is assigned a type such as Hospital, Health Facility, etc. Please list the unique types and count for each.

In [20]:
print( "Number of unique facility types:", df[type_col].nunique())
print(df[type_col].value_counts())

Number of unique facility types: 7
PHC_CENTER          1540
MCH_CARE_CENTER      637
GENERAL_HOSP          97
PRIVATE_FACILITY      20
TEACHING_HOSP          4
SPIRIT_HEALER          1
TRAD_HEALER            1
Name: type_of_facility_visited, dtype: int64


# 8. Data Cleaning

In [21]:
# obtain latitude and longitude from geometry
# Re-project to WGS84
df['geometry'] = df['geometry'].to_crs(epsg=4326)
df['Latitude'] = df['geometry'].y
df['Longitude'] = df['geometry'].x

In [22]:
df.reset_index(inplace=True)

In [23]:
# drop unimportant columns
df.drop(columns=['starttime', 'endtime', 'today', 'deviceid', 'date_of_visit', 
                 'ist', 'countries', 'name_of_facility_visited_other', 
                 'geometry', 'today_dt', 'date_of_visit_dt', 'long', 'lat', 
                 'F27', 'F28', 'F29', 'F_index', 'F_parent_table_name', 
                 'F_parent_index', 'F_tags', 'F_notes', 'F_version', 'F_duration', 
                 'F_submitted_by', 'F_xform_id', 'longitude', 'latitude', 
                 'Weeknumbers', 'YearofActiveSurv', 'geometry', 'today_dt', 
                 'date_of_visit_dt', 'Weeknumbers_new', 'YearofActiveSurv_new'],
       inplace=True)

In [24]:
# priority level, H and Hi both exist.
# change Hi to H, same meaning
df['priority_level'] = df['priority_level'].str.replace('Hi','H')

In [25]:
# states, change WESTERN AREA to Western for consistency
# only uppercase first letter
df['states'] = df['states'].str.title().str.replace('Area', '').str.strip()

In [26]:
# districts
# replace WESTERN RUR with Western Area Rural, WESTERN URB with Western Area Urban for consistency
df['districts'] = df['districts'].str.replace('WESTERN RUR', 'Western Area Rural') \
.str.replace('WESTERN URB', 'Western Area Urban') \
.str.title()

In [27]:
# wards
# uppercase first letter for consistency
# some values contain facility types, remove the types from those values
types = ['CHC', 'CHP']
df['wards'] = df['wards'].str.replace('|'.join(types), '', case=False).str.title().str.strip()

In [28]:
# name_of_facility_visited
# uppercase everything
# strip whitespace
df['name_of_facility_visited'] = df['name_of_facility_visited'].str.upper().str.strip()

In [47]:
# name cleaning
# sort by date
df.sort_values(by=['states', 'districts', 'wards',
                       'name_of_facility_visited', 'date'],
                  inplace=True)

In [63]:
# drop duplicates, identical values in states, districts, wards, facility name
# keep last record
df_dropped = df[~df.duplicated(subset=['states', 'districts', 'wards', 
                         'name_of_facility_visited'],
    keep = 'last')]

In [49]:
print("Number of points dropped:", df.shape[0]-df_dropped.shape[0] )
print("Number of data points remaining:", df_dropped.shape[0])

Number of points dropped: 768
Number of data points remaining: 1532


In [58]:
common_types =  ['CHC', 'MCHP', 'CHP', 'Referral Hospital', 'Government Hospital', 
                 'Mission Hospital', 'Mis Hospital', 'Memorial Hospital', 
                  'Police Hospital', 'Community Hospital',
                 'Military Hospital', 'Government',
                 'Industrial Hospital', 'Hospital', 'Community Health Post',
                'Community Health Centre', 'Health Centre', 'Health Post', 
                 'Memorial Clinic', 'Police Clinic', 'Clinic', 'Dental', 'Mental']

In [64]:
df_dropped['name_short'] = df_dropped[name_col].str.replace('_',' ')\
.str.replace('-',' ')\
.str.replace(',',' ')\
.str.replace('.',' ') \
.str.replace("   ", " ")\
.str.replace("  ", " ")\
.str.replace(' Hosp | Hosp$',' Hospital ', case=False) \
.str.replace(' Gov ',' Government ', case=False) \
.str.replace(' Govt | Govt$',' Government ', case=False) \
.str.replace('underfive|underfives',' Under Five', case=False) \
.str.title() \
.str.replace('|'.join(common_types), '', regex=True, case = False) \
.str.strip()

In [65]:
#df_dropped[['states', 'districts', 'wards', 'name_of_facility_visited', 
#    'name_short', 'type_of_facility_visited']].sample(5)

In [66]:
df_dropped.rename(columns={'states':"Region", 'districts':'District', 
                           'wards':'Chiefdom', 'name_of_facility_visited':'Facility Name', 
                           'type_of_facility_visited':"Facility Type", 
                           'date':"Date", 'name_short':'Short Name',
                          'index':'ID'},inplace=True)
df_dropped.reset_index(drop=True, inplace=True)

In [70]:
#df_dropped[['Region', 'District', 'Chiefdom', 'Facility Name',
#            'Short Name', 'Facility Type']].head()

In [68]:
# further drop duplicates
# identical values in Region, District, Chiefdom, Short Name, Facility Type
df_dropped2 = df_dropped[~df_dropped.duplicated(subset=['Region', 'District', 'Chiefdom', 
                         'Short Name', 'Facility Type'], keep = 'last')]

In [69]:
print("Number of points dropped:", df_dropped.shape[0]-df_dropped2.shape[0] )
print("Number of data points remaining:", df_dropped2.shape[0])

Number of points dropped: 83
Number of data points remaining: 1449


In [72]:
df_dropped2['Source'] = 'ISS'

In [74]:
df_dropped2[df_dropped2.duplicated(subset=['Longitude','Latitude'], keep = False)]

,ID,Region,District,Chiefdom,designation,Facility Name,Facility Type,priority_level,facility_fp_acs,joint_supervision,supervised_by_dsno,supervised_by_who,current_feed_back,surveillance_fp_training,case_def_copy_copy1,facility_fp_acs_copy,physiotherapy_unit,Date,Latitude,Longitude,Short Name,Source


In [76]:
df_dropped2.to_csv(dataDir + "\\Cleaned Data\\Overall\\ISS_cleaned_0814.csv",
                 index=False)

# 8. Make a map of the dataset, how does the data look? Does it all fall within Africa?

Since `geometry` has some missing values, rows with `geometry` missing are dropped. 

It looks like all the data points all fall within the country. 

In [30]:
# Re-project to WGS84
df['geometry'] = df['geometry'].to_crs(epsg=4326)

In [27]:
def make_map(zoom_start, df):
    m = folium.Map(location=[8.542941, -11.783450], 
                   zoom_start=zoom_start, control_scale=True)

    # Create a Clustered map where points are clustered
    marker_cluster = MarkerCluster().add_to(m)
    # Create health facilities as points on top of the map
    for idx, row in df.iterrows():
        # Get lat and lon of points
        lon = row['geometry'].x
        lat = row['geometry'].y
        name = row[name_col]

        # Add marker to the map
        folium.Marker(location=[lat, lon], popup=name).add_to(m) 
    return m

In [35]:
#make_map(zoom_start=7, df=df)

![](maps/ISS_13.png)